In [7]:
from pynq import Overlay
from pynq import MMIO
from pynq import allocate
import numpy as np
import math
import os

print(os.getcwd())
# Program bitstream to FPGA
overlay = Overlay('./DES_axi_wrapper.bit')

# Access to AXI DMA
dma = overlay.axi_dma_0
dma_send = dma.sendchannel
dma_recv = dma.recvchannel

/home/xilinx/jupyter_notebooks/thies/notebooks/DMA


In [8]:
SIZE_OF_ALGO = 8 # 8 bytes = 64 bits
MAX_PARALLEL_SEND = 8

input_type = np.dtype((">V", SIZE_OF_ALGO * 2))
output_type = np.dtype((">V", SIZE_OF_ALGO))
print(f"Using {MAX_PARALLEL_SEND}x type with input: {input_type.itemsize * 8} bits, and output: {output_type.itemsize * 8} bits")

# Allocate physical memory
input_buffer = allocate(shape=(MAX_PARALLEL_SEND,), dtype=input_type)
output_buffer = allocate(shape=(MAX_PARALLEL_SEND,), dtype=output_type)

Using 8x type with input: 128 bits, and output: 64 bits


In [9]:
message = "AAAAAAAA"
key = "BBBBBBBB"


key_bytes = bytes(key, "ascii")
assert(len(key_bytes) == 8)
print(key_bytes.hex())

message_bytes = bytes(message, "ascii")
print(message_bytes.hex())

iterations = math.ceil(len(message_bytes) / SIZE_OF_ALGO)
assert(iterations < MAX_PARALLEL_SEND)

4242424242424242
4141414141414141


In [4]:
# Write input
for i in range(iterations):
    part = message_bytes[8 * i : 8 * (i + 1)]
    input_buffer[i] = ((part + bytes(SIZE_OF_ALGO - len(part))) + key_bytes[0:8])[::-1]
    
    print(input_buffer[i])

b'\x42\x42\x42\x42\x42\x42\x42\x42\x41\x41\x41\x41\x41\x41\x41\x41'


In [13]:
# # Check the written data
for i in range(iterations):
    print("Data: ", np.binary_repr(input_buffer[i]))

TypeError: 'numpy.void' object cannot be interpreted as an integer

In [5]:
# Do AXI DMA MM2S transfer
dma_send.transfer(input_buffer)
# Do AXI DMA S2MM transfer
dma_recv.transfer(output_buffer)

In [6]:
# Check the memory content after DMA transfer
for i in range(iterations):
    print(bytes(output_buffer[i]).hex())

18ca39bbc12efa57


In [10]:

# Delete buffer to prevent memory leak
del input_buffer, output_buffer